### Developing a real stratergy to invest in the stock market

When developing our stratergy, the goal will be to acquire good and useful information about each stock. We then need to check which parameters play a role in explaining variation in the data and which simply contribute noise to the system. Once we have generated and calculated the financial indicators and then optimised the tuning of these params using something like simulated annealing to search the complex solution space we can test and compare our stratergy against the S&P500 real life data. 

Sentiment data could be useful, financial statements even better, unsure how to extract, useful information from these datasets.

Looking for monthly readjustments for our stock portfolio. Further investigation should be carried 
on arbitrage. 

In [209]:
import yfinance as yf
import pandas as pd
import numpy as np
import os

In [210]:
# tickers = ["AAPL", "MSFT", "GOOGL"]

# URL for S&P 500 companies on Wikipedia
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

# Read the first table from the Wikipedia page
sp500_table = pd.read_html(url)[0]

# Extract the ticker symbols
tickers = sp500_table['Symbol'].tolist()
# print(tickers)


start_date = "2015-01-01"
end_date = "2023-12-31"

In [211]:
def download_stock_data(tickers, start_date, end_date):
    stock_data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker')
    return stock_data

data = download_stock_data(tickers, start_date, end_date)

# data_folder = "C:/Users/codyr/Desktop/Coding_/Quant_Trading_Basics"

# data = pd.read_csv("C:/Users/codyr/Desktop/Coding_/Quant_Trading_Basics/AAPL_stock_data.csv")

data.head()

# data.to_csv('AAPL_stock_data.csv', index=True)

[*********************100%***********************]  503 of 503 completed

5 Failed downloads:
['GEV', 'SW', 'SOLV']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2015-01-01 -> 2023-12-31) (Yahoo error = "Data doesn\'t exist for startDate = 1420088400, endDate = 1703998800")')
['BRK.B']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
['BF.B']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2015-01-01 -> 2023-12-31)')


Ticker           MDLZ                                                        \
Price            Open       High        Low      Close  Adj Close    Volume   
Date                                                                          
2015-01-02  36.430000  36.939999  36.270000  36.790001  29.733419   7707700   
2015-01-05  36.709999  36.770000  36.119999  36.240002  29.288921   9621100   
2015-01-06  36.369999  36.650002  35.790001  35.990002  29.086868   7669500   
2015-01-07  36.189999  37.200001  35.970001  37.119999  30.000134  10418800   
2015-01-08  37.090000  37.880001  37.090000  37.590000  30.379974   8122100   

Ticker            AFL                                ...       EPAM  \
Price            Open    High        Low      Close  ...        Low   
Date                                                 ...              
2015-01-02  30.750000  30.805  30.330000  30.540001  ...  46.000000   
2015-01-05  30.219999  30.275  29.620001  29.740000  ...  45.799999   
2015-01-06  29.705000  29.775  29.395000  29.475000  ...  45.770000   
2015-01-07  29.725000  29.980  29.545000  29.660000  ...  46.470001   
2015-01-08  29.870001  30.160  29.730000  29.950001  ...  47.509998   

Ticker                                           MO                        \
Price           Close  Adj Close  Volume       Open       High        Low   
Date                                                                        
2015-01-02  46.470001  46.470001  379300  49.299999  49.639999  48.580002   
2015-01-05  46.270000  46.270000  294300  48.639999  48.990002  48.520000   
2015-01-06  46.290001  46.290001  426200  49.000000  49.650002  48.889999   
2015-01-07  47.290001  47.290001  253900  49.330002  50.029999  49.320000   
2015-01-08  48.669998  48.669998  562300  50.259998  50.930000  50.200001   

Ticker                                     
Price           Close  Adj Close   Volume  
Date                                       
2015-01-02  48.970001  25.831532  6078100  
2015-01-05  48.689999  25.683823  6900200  
2015-01-06  48.980000  25.836790  7240000  
2015-01-07  49.880001  26.311552  5492500  
2015-01-08  50.720001  26.754644  4692600  

[5 rows x 3018 columns]

In [212]:
multi_index_data = (
    data.stack(level=0, future_stack=True)
    .swaplevel()
    .sort_index()
)

multi_index_data

Price                    Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
A      2015-01-02   41.180000   41.310001   40.369999   40.560001   37.353008   
       2015-01-05   40.320000   40.459999   39.700001   39.799999   36.653088   
       2015-01-06   39.810001   40.020000   39.020000   39.180000   36.082130   
       2015-01-07   39.520000   39.810001   39.290001   39.700001   36.561005   
       2015-01-08   40.240002   40.980000   40.180000   40.889999   37.656906   
...                       ...         ...         ...         ...         ...   
ZTS    2023-12-22  195.320007  195.910004  192.740005  194.979996  192.485916   
       2023-12-26  194.880005  196.339996  194.089996  195.500000  192.999283   
       2023-12-27  195.410004  197.009995  194.740005  196.899994  194.381348   
       2023-12-28  197.619995  198.600006  196.529999  197.160004  194.638046   
       2023-12-29  196.679993  198.009995  196.250000  197.369995  194.845352   

Price                 Volume  
Ticker Date                   
A      2015-01-02  1529200.0  
       2015-01-05  2041800.0  
       2015-01-06  2080600.0  
       2015-01-07  3359700.0  
       2015-01-08  2116300.0  
...                      ...  
ZTS    2023-12-22  1548400.0  
       2023-12-26   814600.0  
       2023-12-27   766400.0  
       2023-12-28   880100.0  
       2023-12-29  1007200.0  

[1138792 rows x 6 columns]

In [213]:
### Define and calcuate RSI 
def calculate_rsi(data, column='Adj Close', window=14):
    """
    Calculate the RSI for each stock in a multi-index df
    """
    def rsi_per_stock(data):
    
        if not isinstance(data, pd.DataFrame):
            raise ValueError("Input data must be a pandas DataFrame")
    
        if column not in data.columns:
            raise ValueError(f"Column '{column}' not found in the DataFrame")
    
        # Calculate daily price changes (delta)
        delta = data['Adj Close'].diff()
    
        # Separate gains and losses
        gains = delta.clip(lower=0)
        losses = -delta.clip(upper=0)
        
        avg_gain = gains.rolling(window=window, min_periods=window).mean()
        avg_loss = losses.rolling(window=window, min_periods=window).mean()
    
        rs = avg_gain/avg_loss
        data['RSI'] = 100 - (100/ (1+rs)) # Ratio of avg gains/ avg losses during the window -> normalised to be 0 < rsi < 1
        
        return data
        
    return data.groupby(level='Ticker', group_keys=False).apply(rsi_per_stock)

def calculate_rsi_ewma(data, column="Adj Close", window=14):

    if not isinstance(data, pd.DataFrame):
        raise ValueError("Input data must be a pandas DataFrame")

    if column not in data.columns:
        raise ValueError(f"Column '{column}' not found in the DataFrame")

    def rsi_ewma_per_stock(data):
        # Calculate daily price changes (delta)
        delta = data[column].diff()
    
        # Separate gains and losses
        gains = delta.clip(lower=0)
        losses = -delta.clip(upper=0)
        
        avg_gain = gains.ewm(span=window, adjust=False).mean()
        avg_loss = losses.ewm(span=window, adjust=False).mean()
    
        rs = avg_gain/avg_loss
        rsi = 100 - (100/ (1+rs))

        data['RSI_EWMA'] = 100 - (100/ (1+rs)) # Ratio of exponentially smoothed gains/ losses during the window -> normalised to be 0 < rsi < 1
        return data
        
    return data.groupby(level='Ticker', group_keys=False).apply(rsi_ewma_per_stock)


def calculate_bollinger_bands(data, window=20):
    """
    Calculates Bollinger bands for multi-index df
    """
    if not isinstance(data, pd.DataFrame):
        raise ValueError("Input data must be a pandas DataFrame")
    
    def calculate_bb_per_stock(data):
        bb_data = data['Adj Close']

        rolling_mean = bb_data.rolling(window=window).mean()
        rolling_std = bb_data.rolling(window=window).std()

        data[('Bollinger', 'Middle Band')] = rolling_mean
        data[('Bollinger', 'Upper Band')] = rolling_mean + (2 * rolling_std)
        data[('Bollinger', 'Lower Band')] = rolling_mean - (2 * rolling_std)

        return data

    return data.groupby(level='Ticker', group_keys=False).apply(calculate_bb_per_stock)
    

In [214]:
column = "Adj Close"

df = calculate_rsi(multi_index_data, column)

df = calculate_rsi_ewma(df, column=column, window=14)

df = calculate_bollinger_bands(df)

df.head(25)

Price                   Open       High        Low      Close  Adj Close  \
Ticker Date                                                                
A      2015-01-02  41.180000  41.310001  40.369999  40.560001  37.353008   
       2015-01-05  40.320000  40.459999  39.700001  39.799999  36.653088   
       2015-01-06  39.810001  40.020000  39.020000  39.180000  36.082130   
       2015-01-07  39.520000  39.810001  39.290001  39.700001  36.561005   
       2015-01-08  40.240002  40.980000  40.180000  40.889999  37.656906   
       2015-01-09  41.000000  41.000000  40.290001  40.590000  37.380638   
       2015-01-12  40.610001  40.720001  39.950001  40.110001  36.938599   
       2015-01-13  40.470001  40.700001  39.330002  39.549999  36.422867   
       2015-01-14  39.029999  39.099998  38.209999  39.060001  35.971607   
       2015-01-15  39.330002  39.410000  37.990002  38.009998  35.004646   
       2015-01-16  37.830002  38.459999  37.759998  38.250000  35.225658   
       2015-01-20  38.430000  38.660000  37.759998  37.930000  34.930958   
       2015-01-21  37.750000  38.410000  37.680000  38.160000  35.142773   
       2015-01-22  38.509998  39.860001  38.119999  39.650002  36.514965   
       2015-01-23  39.599998  39.599998  38.759998  38.810001  35.741386   
       2015-01-26  38.790001  39.169998  38.470001  39.150002  36.054493   
       2015-01-27  38.700001  39.250000  38.580002  38.750000  35.686131   
       2015-01-28  39.009998  39.049999  37.959999  38.000000  34.995419   
       2015-01-29  38.000000  38.470001  37.689999  38.459999  35.419052   
       2015-01-30  38.009998  38.320000  37.709999  37.770000  34.783600   
       2015-02-02  37.900002  38.759998  37.709999  38.689999  35.630878   
       2015-02-03  39.040001  39.630001  38.919998  39.619999  36.487343   
       2015-02-04  39.330002  39.459999  38.820000  39.110001  36.017658   
       2015-02-05  39.220001  39.820000  39.220001  39.529999  36.404453   
       2015-02-06  39.540001  39.810001  39.230000  39.340000  36.229469   

Price                 Volume        RSI   RSI_EWMA  (Bollinger, Middle Band)  \
Ticker Date                                                                    
A      2015-01-02  1529200.0        NaN        NaN                       NaN   
       2015-01-05  2041800.0        NaN   0.000000                       NaN   
       2015-01-06  2080600.0        NaN   0.000000                       NaN   
       2015-01-07  3359700.0        NaN   9.739966                       NaN   
       2015-01-08  2116300.0        NaN  28.204983                       NaN   
       2015-01-09  1644900.0        NaN  26.620880                       NaN   
       2015-01-12  2770800.0        NaN  24.119892                       NaN   
       2015-01-13  2013100.0        NaN  21.411858                       NaN   
       2015-01-14  5134000.0        NaN  19.231884                       NaN   
       2015-01-15  2628900.0        NaN  15.364299                       NaN   
       2015-01-16  3004000.0        NaN  19.626986                       NaN   
       2015-01-20  5033600.0        NaN  18.215478                       NaN   
       2015-01-21  2721600.0        NaN  22.818731                       NaN   
       2015-01-22  4856600.0        NaN  45.674800                       NaN   
       2015-01-23  1519300.0  40.374023  38.297500                       NaN   
       2015-01-26  1510900.0  46.251473  42.625389                       NaN   
       2015-01-27  1703500.0  47.455603  38.919662                       NaN   
       2015-01-28  2033500.0  40.207303  32.758069                       NaN   
       2015-01-29  2330000.0  34.716979  39.532702                       NaN   
       2015-01-30  3054300.0  33.093425  33.662737                 36.025946   
       2015-02-02  2508500.0  41.913473  45.998732                 35.939840   
       2015-02-03  2950400.0  50.382578  55.623689                 35.931553   
       2015-02-04  2567400.

In [215]:
def calculate_macd(data, column="Adj Close", fast_period=12, slow_period=26, signal_period=9):
    """
    Calculate MACD, Signal Line, and MACD Histogram for multi-index df
    """
    if not isinstance(data, pd.DataFrame):
        raise ValueError("Input data must be a pandas DataFrame")

    def calculate_macd_per_stock(data):
        macd_data = data['Adj Close']
        
        fast_ema = macd_data.ewm(span=fast_period, adjust=False).mean()
        slow_ema = macd_data.ewm(span=slow_period, adjust=False).mean()
        
        data['MACD Line'] = macd_line = fast_ema - slow_ema
        data['Signal Line'] = signal_line = macd_line.ewm(span=signal_period, adjust=False).mean()

        # Calculate MACD histogram 
        data['Histogram'] = macd_line - signal_line

        return data

    return data.groupby(level='Ticker', group_keys=False).apply(calculate_macd_per_stock)


def calculate_atr(data, window=14):
    """
    Calculate the average true range for multi_index_df
    """
    if not isinstance(data, pd.DataFrame):
        raise ValueError("Input data must be a pandas DataFrame")

    def calculate_atr_per_stock(data):
        # Extract relevant columns
        high = data['High']
        low = data['Low']
        close = data['Adj Close']

        # Calculate True Range (TR)
        true_range = pd.DataFrame({
            'high_low': high - low,
            'high_prev_close': (high - close.shift()).abs(),
            'low_prev_close': (low - close.shift()).abs()
        }).max(axis=1)

        # Calculate ATR using EMA
        data['ATR'] = true_range.ewm(span=window, adjust=False).mean()

        return data

    # Apply calculation per stock
    return data.groupby(level='Ticker', group_keys=False).apply(calculate_atr_per_stock)

def calculate_roc(data, column="Adj Close", periods=14):
    """
    Calculate Rate of Change (RoC) for multi-index DataFrame.
    """
    if not isinstance(data, pd.DataFrame):
        raise ValueError("Input data must be a pandas DataFrame")

    def calculate_roc_per_stock(data):
        # Extract relevant column
        roc_data = data[column]
        
        # Calculate RoC
        data['RoC'] = ((roc_data - roc_data.shift(periods)) / roc_data.shift(periods)) * 100

        return data

    # Apply calculation per stock
    return data.groupby(level='Ticker', group_keys=False).apply(calculate_roc_per_stock)

# Might be something wrong with OBV looks oddly large numbers
def calculate_obv(data):
    """
    Calculate On-Balance Volume (OBV) for a multi-index DataFrame.
    """
    if not isinstance(data, pd.DataFrame):
        raise ValueError("Input data must be a pandas DataFrame")

    def calculate_obv_per_stock(data):
        # Extract relevant columns
        close = data['Close']
        volume = data['Volume']

        # Calculate OBV
        obv = (volume * np.sign(close.diff())).fillna(0).cumsum()

        # Add OBV to the DataFrame
        data['OBV'] = obv

        return data

    # Apply calculation per stock
    return data.groupby(level='Ticker', group_keys=False).apply(calculate_obv_per_stock)

def calculate_moving_averages(data, column="Adj Close", sma_window=50, ema_window=50):
    """
    Calculate SMA and EMA for a MultiIndex DataFrame.
    """
    if not isinstance(data, pd.DataFrame):
        raise ValueError("Input data must be a pandas DataFrame")

    def calculate_averages_per_stock(data):
        # Extract relevant column
        price_data = data[column]
        
        # Calculate SMA
        data[f"SMA_{sma_window}"] = price_data.rolling(window=sma_window).mean()
        
        # Calculate EMA
        data[f"EMA_{ema_window}"] = price_data.ewm(span=ema_window, adjust=False).mean()

        return data

    # Apply calculation per stock
    return data.groupby(level='Ticker', group_keys=False).apply(calculate_averages_per_stock)

def calculate_garman_klass_volatility(data):
    """
    Calculate Garman-Klass Volatility for a MultiIndex DataFrame.
    """
    if not isinstance(data, pd.DataFrame):
        raise ValueError("Input data must be a pandas DataFrame")

    def calculate_gk_per_stock(data):
        # Extract relevant columns
        high = data['High']
        low = data['Low']
        close = data['Close']
        open_price = data['Open']

        # Calculate Garman-Klass Volatility
        gk_volatility = (
            0.5 * (np.log(high / low) ** 2)
            - (2 * np.log(2) - 1) * (np.log(close / open_price) ** 2)
        )

        # Add Garman-Klass Volatility to the DataFrame
        data['GK Volatility'] = np.sqrt(gk_volatility)

        return data

    # Apply calculation per stock
    return data.groupby(level='Ticker', group_keys=False).apply(calculate_gk_per_stock)

In [216]:
df = calculate_macd(df)

df = calculate_atr(df)

df = calculate_roc(df)

df = calculate_obv(df)

df = calculate_moving_averages(df)

df = calculate_garman_klass_volatility(df)

df.tail()

Price                    Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
ZTS    2023-12-22  195.320007  195.910004  192.740005  194.979996  192.485916   
       2023-12-26  194.880005  196.339996  194.089996  195.500000  192.999283   
       2023-12-27  195.410004  197.009995  194.740005  196.899994  194.381348   
       2023-12-28  197.619995  198.600006  196.529999  197.160004  194.638046   
       2023-12-29  196.679993  198.009995  196.250000  197.369995  194.845352   

Price                 Volume        RSI   RSI_EWMA  (Bollinger, Middle Band)  \
Ticker Date                                                                    
ZTS    2023-12-22  1548400.0  69.699843  63.196990                185.167747   
       2023-12-26   814600.0  75.822825  64.648109                185.992560   
       2023-12-27   766400.0  75.050504  68.505574                186.976313   
       2023-12-28   880100.0  75.695634  69.225234                188.031146   
       2023-12-29  1007200.0  73.414034  69.866848                189.052907   

Price              (Bollinger, Upper Band)  (Bollinger, Lower Band)  \
Ticker Date                                                           
ZTS    2023-12-22               202.001140               168.334354   
       2023-12-26               202.654044               169.331076   
       2023-12-27               203.148155               170.804471   
       2023-12-28               203.236489               172.825804   
       2023-12-29               203.107187               174.998626   

Price              MACD Line  Signal Line  Histogram       ATR       RoC  \
Ticker Date                                                                
ZTS    2023-12-22   5.901429     5.667631   0.233798  5.040610  7.061284   
       2023-12-26   5.692417     5.672588   0.019828  4.882406  8.822715   
       2023-12-27   5.574040     5.652879  -0.078839  4.766180  8.186807   
       2023-12-28   5.438250     5.609953  -0.171703  4.693177  8.430945   
       2023-12-29   5.286425     5.545247  -0.258823  4.517013  6.917654   

Price                      OBV      SMA_50      EMA_50  GK Volatility  
Ticker Date                                                            
ZTS    2023-12-22  242159100.0  173.708631  179.299542       0.011484  
       2023-12-26  242973700.0  174.133851  179.836787       0.007907  
       2023-12-27  243740100.0  174.570956  180.407162       0.006698  
       2023-12-28  244620200.0  175.031316  180.965236       0.007266  
       2023-12-29  245627400.0  175.588792  181.509554       0.005926

In [217]:
def calculate_rolling_beta(data, market_column='MArket Pct Change', stock_column="Adj Close", window=30):
    """
    Calculate Rolling Beta for a MultiIndex DataFrame.

    The rolling Beta feature measures a stock's sensitivity to the market's movements over a rolling window
    """
    if not isinstance(data, pd.DataFrame):
        raise ValueError("Input data must be a pandas DataFrame")

    def calculate_beta_per_stock(data):
        # Calculate daily stock and market returns
        stock_returns = data[stock_column].pct_change()
        market_returns = data[market_column]

        # Rolling covariance and variance
        rolling_cov = stock_returns.rolling(window=window).cov(market_returns)
        rolling_var = market_returns.rolling(window=window).var()

        # Calculate Rolling Beta
        data['Rolling Beta'] = rolling_cov / rolling_var

        return data

    # Apply calculation per stock
    return data.groupby(level='Ticker', group_keys=False).apply(calculate_beta_per_stock)

In [218]:
# Define the market index (e.g., S&P 500)
market_ticker = "^GSPC"

# Fetch historical data
market_data = yf.download(market_ticker, start="2015-01-01", end="2023-12-31", progress=False)

market_data['Market Pct Change'] = market_data['Adj Close'].pct_change()

# Get unique tickers from stock data
unique_tickers = df.index.get_level_values('Ticker').unique()

# Expand the market data for each ticker

# Creates the multi-index for the df
market_data_expanded = pd.DataFrame(
    index=pd.MultiIndex.from_product(
        [unique_tickers, market_data.index], 
        names=['Ticker', 'Date']
    )
)

# Replicates the data for however many unique tickers there are
market_data_expanded['Market Pct Change'] = np.tile(
    market_data['Market Pct Change'].values, len(unique_tickers)
)

# Combines this data now with our stock data
df = df.merge(
    market_data_expanded,
    left_index=True,
    right_index=True,
    how='left'
)

# print(df.groupby(level='Ticker')['Market Pct Change'].head())
# df = df.drop(columns='Market Returns')

In [219]:
df = calculate_rolling_beta(df, market_column='Market Pct Change', stock_column='Adj Close')

C:\Users\codyr\AppData\Local\Temp\ipykernel_15268\736523549.py:12: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  stock_returns = data[stock_column].pct_change()
C:\Users\codyr\AppData\Local\Temp\ipykernel_15268\736523549.py:12: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  stock_returns = data[stock_column].pct_change()
C:\Users\codyr\AppData\Local\Temp\ipykernel_15268\736523549.py:12: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method

In [220]:
df.head()

Open       High        Low      Close  Adj Close  \
Ticker Date                                                                
A      2015-01-02  41.180000  41.310001  40.369999  40.560001  37.353008   
       2015-01-05  40.320000  40.459999  39.700001  39.799999  36.653088   
       2015-01-06  39.810001  40.020000  39.020000  39.180000  36.082130   
       2015-01-07  39.520000  39.810001  39.290001  39.700001  36.561005   
       2015-01-08  40.240002  40.980000  40.180000  40.889999  37.656906   

                      Volume  RSI   RSI_EWMA  (Bollinger, Middle Band)  \
Ticker Date                                                              
A      2015-01-02  1529200.0  NaN        NaN                       NaN   
       2015-01-05  2041800.0  NaN   0.000000                       NaN   
       2015-01-06  2080600.0  NaN   0.000000                       NaN   
       2015-01-07  3359700.0  NaN   9.739966                       NaN   
       2015-01-08  2116300.0  NaN  28.204983                       NaN   

                   (Bollinger, Upper Band)  ...  Signal Line  Histogram  \
Ticker Date                                 ...                           
A      2015-01-02                      NaN  ...     0.000000   0.000000   
       2015-01-05                      NaN  ...    -0.011167  -0.044667   
       2015-01-06                      NaN  ...    -0.037831  -0.106658   
       2015-01-07                      NaN  ...    -0.065085  -0.109015   
       2015-01-08                      NaN  ...    -0.073647  -0.034247   

                        ATR  RoC        OBV  SMA_50     EMA_50  GK Volatility  \
Ticker Date                                                                     
A      2015-01-02  0.940002  NaN        0.0     NaN  37.353008       0.013267   
       2015-01-05  1.228934  NaN -2041800.0     NaN  37.325560       0.010710   
       2015-01-06  1.513998  NaN -4122400.0     NaN  37.276798       0.014895   
       2015-01-07  1.809181  NaN  -762700.0     NaN  37.248728       0.008858   
       2015-01-08  2.157156  NaN  1353600.0     NaN  37.264735       0.009754   

                   Market Pct Change  Rolling Beta  
Ticker Date                                         
A      2015-01-02                NaN           NaN  
       2015-01-05          -0.018278           NaN  
       2015-01-06          -0.008893           NaN  
       2015-01-07           0.011630           NaN  
       2015-01-08           0.017888           NaN  

[5 rows x 22 columns]

### Looking at influential figures and politicians investments

#### This area only provides us access to very recent trades with little historical data to back it up. Might be used later for re-balancing our portfolio

In [221]:
"""

import requests
from bs4 import BeautifulSoup

# url = "https://www.quiverquant.com/congresstrading/"

# response = requests.get(url)
# if response.status_code == 200:
#     print("Successfully fetched webpage content!")
#     html_content = response.content
# else:
#     print(f"Failed to fetch webpage. Status code: {response.status_code}")

#Parse the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

table = soup.find('table')

if table:
    print("Table found!")
else:
    print("No table found")

"""

'\n\nimport requests\nfrom bs4 import BeautifulSoup\n\n# url = "https://www.quiverquant.com/congresstrading/"\n\n# response = requests.get(url)\n# if response.status_code == 200:\n#     print("Successfully fetched webpage content!")\n#     html_content = response.content\n# else:\n#     print(f"Failed to fetch webpage. Status code: {response.status_code}")\n\n#Parse the HTML content\nsoup = BeautifulSoup(html_content, \'html.parser\')\n\ntable = soup.find(\'table\')\n\nif table:\n    print("Table found!")\nelse:\n    print("No table found")\n\n'

In [222]:
"""
# Extract all rows from the table
rows = table.find_all('tr')

# Loop through rows and extract cell data
data = []
for row in rows[1:]:
    cells = row.find_all('td')
    row_data = [cell.text.strip() for cell in cells] # Get text from each cell without whitespace
    data.append(row_data)

# print("Extracted data:", data[:5])

cleaned_data = []
for row in data:
    # Clean each cell in the row by stripping newlines and excessive spaces
    cleaned_row = [cell.replace("\n", "").replace("  ", " ").strip() for cell in row]
    cleaned_row = [" ".join(cell.split()) for cell in row]
    cleaned_data.append(cleaned_row)

print("Cleaned Data:", cleaned_data[:5])

"""

'\n# Extract all rows from the table\nrows = table.find_all(\'tr\')\n\n# Loop through rows and extract cell data\ndata = []\nfor row in rows[1:]:\n    cells = row.find_all(\'td\')\n    row_data = [cell.text.strip() for cell in cells] # Get text from each cell without whitespace\n    data.append(row_data)\n\n# print("Extracted data:", data[:5])\n\ncleaned_data = []\nfor row in data:\n    # Clean each cell in the row by stripping newlines and excessive spaces\n    cleaned_row = [cell.replace("\n", "").replace("  ", " ").strip() for cell in row]\n    cleaned_row = [" ".join(cell.split()) for cell in row]\n    cleaned_data.append(cleaned_row)\n\nprint("Cleaned Data:", cleaned_data[:5])\n\n'

In [223]:
"""
# Extract column names from the header row
header_row = rows[0]

headers = [th.text.strip() for th in header_row.find_all('th')]

print("Headers:", headers)
"""

'\n# Extract column names from the header row\nheader_row = rows[0]\n\nheaders = [th.text.strip() for th in header_row.find_all(\'th\')]\n\nprint("Headers:", headers)\n'

In [224]:
"""
import pandas as pd

# Refine our data cleaning to obtain singular elements
refined_data = []
for row in cleaned_data:
    # Split the Stock Column into Ticker and Company Name
    stock_parts = row[0].split(maxsplit=1)
    ticker = stock_parts[0].strip()
    company = stock_parts[1].strip() if len(stock_parts) > 1 else ""

    # Split the Transaction Column into Type and Amount Range
    transaction_parts = row[1].split("$", 1)
    transaction_type = transaction_parts[0].strip()
    # amount_range = "$" + transaction_parts[1].strip() if len(transaction_parts)>1 else ""
    if len(transaction_parts) > 1:
        # Further split the amount range to capture both numbers
        amount_range_parts = transaction_parts[1].split("-", 1)
        lower_amount = amount_range_parts[0].replace(",", "").strip()
        upper_amount = amount_range_parts[1].replace(",", "").strip() if len(amount_range_parts)>1 else ""
        upper_amount = upper_amount.replace("$", "").strip()
        amount_range = f"{lower_amount} - {upper_amount}"
    else:
        amount_range = ""
    
    # Split the Transaction Column into Type and Amount Range
    if "Senate" in row[2] or "House" in row[2]:
        split_point = row[2].find("Senate") if "Senate" in row[2] else row[2].find("House")
        politician_name = row[2][:split_point].strip()
        politician_affiliation = row[2][split_point:].strip()
    else:
        politician_name = row[2].strip()
        politican_affiliation = ""

    # Clean and parse the performance column
    performance = row[6].replace("%", "").strip()
    performance = float(performance) if performance not in ["-", ""] else None

    refined_data.append([
    ticker, company, transaction_type, amount_range, 
    politician_name, politician_affiliation, 
    row[3], row[4], performance
    ])

columns = [
    "Ticker", "Company", "Transaction Type", "Amount Range", 
    "Politician", "Affiliation", 
    "Reported Date", "Transaction Date", "Performance"
]

# Convert our data in a pandas df
politician_df = pd.DataFrame(refined_data, columns=columns)

# Convert dates to datetime format
politician_df["Reported Date"] = pd.to_datetime(df["Reported Date"], errors="coerce")
politician_df["Transaction Date"] = pd.to_datetime(df["Transaction Date"], errors="coerce")


politician_df.head()
"""

'\nimport pandas as pd\n\n# Refine our data cleaning to obtain singular elements\nrefined_data = []\nfor row in cleaned_data:\n    # Split the Stock Column into Ticker and Company Name\n    stock_parts = row[0].split(maxsplit=1)\n    ticker = stock_parts[0].strip()\n    company = stock_parts[1].strip() if len(stock_parts) > 1 else ""\n\n    # Split the Transaction Column into Type and Amount Range\n    transaction_parts = row[1].split("$", 1)\n    transaction_type = transaction_parts[0].strip()\n    # amount_range = "$" + transaction_parts[1].strip() if len(transaction_parts)>1 else ""\n    if len(transaction_parts) > 1:\n        # Further split the amount range to capture both numbers\n        amount_range_parts = transaction_parts[1].split("-", 1)\n        lower_amount = amount_range_parts[0].replace(",", "").strip()\n        upper_amount = amount_range_parts[1].replace(",", "").strip() if len(amount_range_parts)>1 else ""\n        upper_amount = upper_amount.replace("$", "").strip()

### Loading Historical Political trades

## Need to adjust for survivorship bias eventually

In [225]:
import os

data_folder = "C:/Users/codyr/Desktop/Coding_/Quant_Trading_Basics/Data"

political_data = pd.read_csv(os.path.join(data_folder, 'politician_trading_data_2020.csv'))

political_data['Transaction Date'] = pd.to_datetime(political_data['transaction_date'], errors='coerce')

political_data = political_data.set_index('Transaction Date')

political_data = political_data.sort_index()

political_data = political_data[(political_data.index >= '2019-01-01') & (political_data.index <= '2023-05-03')]

sp500_tickers = pd.read_csv('https://datahub.io/core/s-and-p-500-companies/r/constituents.csv')['Symbol'].tolist()

political_data = political_data[political_data['ticker'].isin(sp500_tickers)]

print(f"Filtered political data:\n\n{political_data.head()}")

Filtered political data:

                  disclosure_year disclosure_date transaction_date owner  \
Transaction Date                                                           
2019-01-09                   2020      04/27/2020       2019-01-09    --   
2019-01-09                   2020      04/27/2020       2019-01-09    --   
2019-01-23                   2020      04/27/2020       2019-01-23    --   
2019-01-23                   2020      04/27/2020       2019-01-23    --   
2019-01-23                   2020      04/27/2020       2019-01-23    --   

                 ticker             asset_description          type  \
Transaction Date                                                      
2019-01-09         DECK   Deckers Outdoor Corporation  sale_partial   
2019-01-09           RL      Ralph Lauren Corporation  sale_partial   
2019-01-23          AMD  Advanced Micro Devices, Inc.      purchase   
2019-01-23         DECK   Deckers Outdoor Corporation  sale_partial   
2019-01-23     

### Fetch historical data to determine what the adj closing price was on the transaction date

In [226]:
import yfinance as yf

sp500_data = yf.download(sp500_tickers,start="2019-01-09",end="2023-05-03", group_by='ticker')

reshaped_data = sp500_data.stack(level=0, future_stack=True).swaplevel().sort_index()

filtered_stock_data = reshaped_data.loc[political_data['ticker'].unique()]

filtered_stock_data = filtered_stock_data['Adj Close']
filtered_stock_data.head()

filtered_stock_data = filtered_stock_data.reset_index()
filtered_stock_data.head()

political_data = political_data.reset_index()
political_data = political_data[["Transaction Date", "ticker", "type", "amount", "representative"]]
political_data = political_data.rename(columns={"Transaction Date": "Date", "ticker": "Ticker", "type": "Type", "amount": "Amount", "representative": "Representative"})
political_data.head()

# Merge the two datasets
merged_data = pd.merge(
    political_data,
    filtered_stock_data,
    on=['Ticker', 'Date'],
    how="inner"
)

merged_data = merged_data.set_index(["Ticker", "Date"])

merged_data.head()


[*********************100%***********************]  503 of 503 completed

7 Failed downloads:
['KVUE', 'GEV', 'SW', 'SOLV', 'VLTO']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2019-01-09 -> 2023-05-03) (Yahoo error = "Data doesn\'t exist for startDate = 1547010000, endDate = 1683086400")')
['BRK.B']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
['BF.B']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2019-01-09 -> 2023-05-03)')


,,Type,Amount,Representative,Adj Close
Ticker,Date,,,,
DECK,2019-01-09,sale_partial,"$1,001 - $15,000",Donna Shalala,21.103333
RL,2019-01-09,sale_partial,"$1,001 - $15,000",Donna Shalala,96.827095
AMD,2019-01-23,purchase,"$1,001 - $15,000",Donna Shalala,19.799999
DECK,2019-01-23,sale_partial,"$1,001 - $15,000",Donna Shalala,20.591667
PNR,2019-01-23,purchase,"$1,001 - $15,000",Donna Shalala,36.596737


In [227]:
# Define the mapping for amount ranges to midpoint values
amount_mapping = {
    '$1,001 - $15,000': 8000.0,
    '$15,001 - $50,000': 32500.0,
    '$50,001 - $100,000': 75000.0,
    '$100,001 - $250,000': 175000.0,
    '$1,001 -': 1001.0,  # Assume the minimum for open-ended range
    '$1,000,001 - $5,000,000': 3000000.0,
    '$250,001 - $500,000': 375000.0,
    '$500,001 - $1,000,000': 750000.0,
    '$5,000,001 - $25,000,000': 15000000.0,
    '$15,000 - $50,000': 32500.0,  # Align with similar range
    '$1,000 - $15,000': 8000.0     # Align with similar range
}

# Map the amount ranges to their midpoint values
merged_data['Amount'] = merged_data['Amount'].map(amount_mapping)

# Check the updated data
print(merged_data['Amount'].head(30))
print(merged_data['Amount'].isna().sum())

Ticker  Date      
DECK    2019-01-09     8000.0
RL      2019-01-09     8000.0
AMD     2019-01-23     8000.0
DECK    2019-01-23     8000.0
PNR     2019-01-23     8000.0
CAT     2019-01-29    32500.0
CL      2019-01-29     8000.0
KHC     2019-02-01     8000.0
CVX     2019-02-05     8000.0
VZ      2019-02-05     8000.0
AAPL    2019-02-05     8000.0
PFE     2019-02-05     8000.0
MSFT    2019-02-05     8000.0
BK      2019-02-05     8000.0
CB      2019-02-05     8000.0
COP     2019-02-05     8000.0
NEE     2019-02-05     8000.0
CSCO    2019-02-05     8000.0
LMT     2019-02-05     8000.0
WMT     2019-02-05     8000.0
MCD     2019-02-05     8000.0
LOW     2019-02-05     8000.0
JNJ     2019-02-05     8000.0
SRE     2019-02-05     8000.0
CMCSA   2019-02-05     8000.0
WFC     2019-02-05     8000.0
HPQ     2019-02-05     8000.0
CL      2019-02-05     8000.0
OXY     2019-02-05     8000.0
JPM     2019-02-05     8000.0
Name: Amount, dtype: float64
0


In [228]:
merged_data['Type'].unique()

array(['sale_partial', 'purchase', 'sale_full', 'exchange', 'sale'],
      dtype=object)

In [229]:
# Add a column for Quarter
merged_data['Quater'] = merged_data.index.get_level_values('Date').to_period('Q')

def categorise_transaction_type(transaction_type):
    transaction_type = transaction_type.lower()
    if 'purchase' in transaction_type:
        return 'buy'
    elif'sale' in transaction_type:
        return 'sell'

merged_data['Transaction Category'] = merged_data['Type'].apply(categorise_transaction_type)

# Group by Ticker and Quarter
quarterly_aggregated = merged_data.groupby(['Ticker', 'Quater']).agg(
    Total_Transactions=('Amount', 'count'),
    Total_Buy_Amount=(
        'Amount',
        lambda x: sum(
            amt for amt, trans_type in zip(x, merged_data.loc[x.index, 'Transaction Category']) if trans_type == 'buy'
        )
    ),
    Total_Sell_Amount=(
        'Amount',
        lambda x: sum(
            amt for amt, trans_type in zip(x, merged_data.loc[x.index, 'Transaction Category']) if trans_type == 'sell'
        )
    )
).reset_index()

# Calculate Net Buy-Sell Amount
quarterly_aggregated['Net_Buy_Sell'] = (
    quarterly_aggregated['Total_Buy_Amount'] - quarterly_aggregated['Total_Sell_Amount']
)

# Display the aggregated data
print(quarterly_aggregated.head())

  Ticker  Quater  Total_Transactions  Total_Buy_Amount  Total_Sell_Amount  \
0      A  2020Q2                   1               0.0             8000.0   
1      A  2021Q1                   1               0.0             8000.0   
2   AAPL  2019Q1                   1               0.0             8000.0   
3   AAPL  2019Q3                   3               0.0            48500.0   
4   AAPL  2019Q4                  12           48500.0           145500.0   

   Net_Buy_Sell  
0       -8000.0  
1       -8000.0  
2       -8000.0  
3      -48500.0  
4      -97000.0  


In [230]:
# Only include stocks in quaters which are traded more than 5 times. 
quarterly_aggregated = quarterly_aggregated[quarterly_aggregated['Total_Transactions']>=5]

In [231]:
import numpy as np

# Compute the Buy/Sell ratio
epsilon = 1e-6
quarterly_aggregated['Buy_Sell_Ratio'] = (quarterly_aggregated['Total_Buy_Amount'] + epsilon)/ (quarterly_aggregated['Total_Sell_Amount'] + epsilon)

# Logarithmic transformation to ease mean and std calculation
quarterly_aggregated['Log_Buy_Sell_Ratio'] = np.log(quarterly_aggregated['Buy_Sell_Ratio'])

# Calculate mean and standard deviation
mean_log_ratio = quarterly_aggregated['Log_Buy_Sell_Ratio'].mean()
std_log_ratio = quarterly_aggregated['Log_Buy_Sell_Ratio'].std()

print(f"Mean Log Ratio: {mean_log_ratio:.4f}")
print(f"Standard Deviation of Log Ratio: {std_log_ratio:.4f}")

# Calculate z-score
quarterly_aggregated['Z_Score'] = (quarterly_aggregated['Log_Buy_Sell_Ratio'] - mean_log_ratio) / std_log_ratio

# Function to assign signals
def assign_signal(z):
    if z >= 2:
        return 'Strong Buy'
    elif 1 <= z < 2:
        return 'Buy'
    elif 0 < z < 1:
        return 'Weak Buy - Hold'
    elif -1 < z < 0:
        return 'Weak Sell - Hold'
    elif -2 < z <= -1:
        return 'Sell'
    else:
        return 'Strong Sell'

# Apply the function to assign signals
quarterly_aggregated['Signal'] = quarterly_aggregated['Z_Score'].apply(assign_signal)

# Calculate confidence score
quarterly_aggregated['Confidence'] = quarterly_aggregated['Z_Score'].abs()

print(quarterly_aggregated[['Ticker', 'Quater', 'Total_Buy_Amount', 'Total_Sell_Amount', 'Buy_Sell_Ratio', 'Log_Buy_Sell_Ratio', 'Z_Score', 'Signal', 'Confidence']].head())

Mean Log Ratio: 0.8755
Standard Deviation of Log Ratio: 11.1354
  Ticker  Quater  Total_Buy_Amount  Total_Sell_Amount  Buy_Sell_Ratio  \
4   AAPL  2019Q4           48500.0           145500.0        0.333333   
5   AAPL  2020Q1         1011000.0          7113501.0        0.142124   
6   AAPL  2020Q2         4679500.0          3129000.0        1.495526   
7   AAPL  2020Q3           41501.0           315500.0        0.131540   
8   AAPL  2020Q4          170000.0          1254500.0        0.135512   

   Log_Buy_Sell_Ratio   Z_Score            Signal  Confidence  
4           -1.098612 -0.177282  Weak Sell - Hold    0.177282  
5           -1.951055 -0.253835  Weak Sell - Hold    0.253835  
6            0.402478 -0.042479  Weak Sell - Hold    0.042479  
7           -2.028441 -0.260784  Weak Sell - Hold    0.260784  
8           -1.998694 -0.258113  Weak Sell - Hold    0.258113  


In [232]:
quarterly_aggregated.rename(columns={'Quater':'Quarter'}, inplace=True)

quarterly_aggregated.head()

,Ticker,Quarter,Total_Transactions,Total_Buy_Amount,Total_Sell_Amount,Net_Buy_Sell,Buy_Sell_Ratio,Log_Buy_Sell_Ratio,Z_Score,Signal,Confidence
4,AAPL,2019Q4,12,48500.0,145500.0,-97000.0,0.333333,-1.098612,-0.177282,Weak Sell - Hold,0.177282
5,AAPL,2020Q1,32,1011000.0,7113501.0,-6102501.0,0.142124,-1.951055,-0.253835,Weak Sell - Hold,0.253835
6,AAPL,2020Q2,25,4679500.0,3129000.0,1550500.0,1.495526,0.402478,-0.042479,Weak Sell - Hold,0.042479
7,AAPL,2020Q3,21,41501.0,315500.0,-273999.0,0.131540,-2.028441,-0.260784,Weak Sell - Hold,0.260784
8,AAPL,2020Q4,18,170000.0,1254500.0,-1084500.0,0.135512,-1.998694,-0.258113,Weak Sell - Hold,0.258113


### Building out an initial portfolio from historical stock data alone

In [233]:
df.rename(columns={('Bollinger', 'Middle Band'): 'Middle Band', ('Bollinger', 'Lower Band'): 'Lower Band', ('Bollinger', 'Upper Band'): 'Upper Band'}, inplace=True)

# Dropping NA's
df = df.groupby('Ticker', group_keys=False).apply(lambda x: x.iloc[50:])
df.reset_index()

df.head()

Open       High        Low      Close  Adj Close  \
Ticker Date                                                                
A      2015-03-17  41.520000  41.689999  41.060001  41.580002  38.292370   
       2015-03-18  41.450001  42.380001  41.169998  42.119999  38.789661   
       2015-03-19  42.049999  42.250000  41.820000  42.209999  38.872547   
       2015-03-20  42.389999  42.529999  42.200001  42.209999  38.872547   
       2015-03-23  42.119999  42.439999  42.000000  42.200001  38.863327   

                      Volume        RSI   RSI_EWMA  Middle Band  Upper Band  \
Ticker Date                                                                   
A      2015-03-17  1504900.0  44.464498  52.899729    38.425435   39.469240   
       2015-03-18  1295600.0  47.993345  60.496175    38.452142   39.504937   
       2015-03-19  1085700.0  50.000000  61.684614    38.500952   39.535613   
       2015-03-20  2315500.0  45.487999  61.684614    38.512924   39.559495   
       2015-03-23  1835400.0  49.399877  61.411002    38.534565   39.591806   

                   ...  Signal Line  Histogram       ATR       RoC        OBV  \
Ticker Date        ...                                                          
A      2015-03-17  ...     0.377676  -0.120665  3.538270 -1.469141   931800.0   
       2015-03-18  ...     0.359830  -0.071383  3.611518 -0.566562  2227400.0   
       2015-03-19  ...     0.351144  -0.034743  3.591361  0.000000  3313100.0   
       2015-03-20  ...     0.347855  -0.013157  3.600173 -1.147548  3313100.0   
       2015-03-23  ...     0.347181  -0.002698  3.595810 -0.142008  1477700.0   

                      SMA_50     EMA_50  GK Volatility  Market Pct Change  \
Ticker Date                                                                 
A      2015-03-17  37.045972  37.541117       0.010730          -0.003320   
       2015-03-18  37.088703  37.590080       0.017895           0.012158   
       2015-03-19  37.144511  37.640373       0.006838          -0.004873   
       2015-03-20  37.190742  37.688693       0.004831           0.009013   
       2015-03-23  37.214871  37.734757       0.007274          -0.001746   

                   Rolling Beta  
Ticker Date                      
A      2015-03-17      1.435489  
       2015-03-18      1.375154  
       2015-03-19      1.320370  
       2015-03-20      1.275087  
       2015-03-23      1.269052  

[5 rows x 22 columns]

In [234]:
from sklearn.preprocessing import StandardScaler

# Select features for normalization
features = ['Volume', 'RSI', 'RSI_EWMA', 'MACD Line', 'ATR', 'Middle Band', 'Upper Band', 'Lower Band', 'Signal Line', 'Histogram', 'RoC',  'OBV', 'SMA_50', 'EMA_50', 'GK Volatility', 'Market Pct Change', 'Rolling Beta']
df_features = df[features]

scaler = StandardScaler()
df_normalised = pd.DataFrame(scaler.fit_transform(df_features), columns=features, index=df.index)

df_normalised.head()

Volume       RSI  RSI_EWMA  MACD Line       ATR  \
Ticker Date                                                            
A      2015-03-17 -0.171102 -0.503458 -0.000724  -0.025527 -0.604414   
       2015-03-18 -0.178605 -0.294759  0.462353  -0.019060 -0.597702   
       2015-03-19 -0.186130 -0.176084  0.534800  -0.013310 -0.599549   
       2015-03-20 -0.142043 -0.442927  0.534800  -0.009546 -0.598741   
       2015-03-23 -0.159254 -0.211576  0.518120  -0.007533 -0.599141   

                   Middle Band  Upper Band  Lower Band  Signal Line  \
Ticker Date                                                           
A      2015-03-17    -0.346009   -0.351238   -0.339529     0.000678   
       2015-03-18    -0.345888   -0.351086   -0.339444    -0.003270   
       2015-03-19    -0.345667   -0.350954   -0.339121    -0.005192   
       2015-03-20    -0.345612   -0.350852   -0.339121    -0.005920   
       2015-03-23    -0.345514   -0.350714   -0.339068    -0.006069   

                   Histogram       RoC       OBV    SMA_50    EMA_50  \
Ticker Date                                                            
A      2015-03-17  -0.083036 -0.315258 -0.093685 -0.352246 -0.349892   
       2015-03-18  -0.050854 -0.194259 -0.093202 -0.352050 -0.349668   
       2015-03-19  -0.026927 -0.118306 -0.092797 -0.351794 -0.349437   
       2015-03-20  -0.012832 -0.272145 -0.092797 -0.351583 -0.349215   
       2015-03-23  -0.006002 -0.137343 -0.093481 -0.351472 -0.349004   

                   GK Volatility  Market Pct Change  Rolling Beta  
Ticker Date                                                        
A      2015-03-17      -0.370571          -0.324785      0.781444  
       2015-03-18       0.351412           1.011437      0.677310  
       2015-03-19      -0.762746          -0.458799      0.582756  
       2015-03-20      -0.964887           0.739881      0.504601  
       2015-03-23      -0.718739          -0.188868      0.494184

Next steps:
- We need to focus on actually contriving some sort of model which can predict which stocks to include in portfolio.
- We need to offset the dates so that the current month decides the portfolio makeup for next month

## Constructing an initial portfolio

### Transformations

- Log Transformation for Volume – Prevents massive differences between small and large-cap stocks.
- Z-score for Technical Indicators – Ensures comparability by standardizing mean and variance.
- MinMax for OBV – Since OBV accumulates, MinMax scaling ensures it's relative across stocks.
- No Scaling for Market % Change & Beta – As they are already in a comparable format.

In [235]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Define feature groups based on their characteristics
z_score_features = ['ATR', 'GK Volatility', 'RSI', 'RSI_EWMA', 'MACD Line', 
                    'Signal Line', 'Histogram', 'RoC', 'SMA_50', 'EMA_50', 
                    'Middle Band', 'Upper Band', 'Lower Band']
minmax_features = ['OBV']  # MinMax ensures OBV stays within 0-1
log_features = ['Volume']  # Log transformation for highly skewed data
no_scaling_features = ['Market Pct Change', 'Rolling Beta']

# Apply Standard Scaling (Z-score normalization)
scaler_std = StandardScaler()
df_standardized = pd.DataFrame(scaler_std.fit_transform(df[z_score_features]), 
                               columns=z_score_features, index=df.index)

# Apply MinMax Scaling (0 to 1)
scaler_minmax = MinMaxScaler()
df_minmax = pd.DataFrame(scaler_minmax.fit_transform(df[minmax_features]), 
                         columns=minmax_features, index=df.index)

# Apply Log transformation (log(1 + x) to handle zero values)
df_log = df.copy()
df_log[log_features] = np.log1p(df[log_features])

# Keep unscaled features
df_transformed = df.copy()
df_transformed[z_score_features] = df_standardized
df_transformed[minmax_features] = df_minmax
df_transformed[log_features] = df_log[log_features]
df_transformed[no_scaling_features] = df[no_scaling_features]

df_transformed.head()


Open       High        Low      Close  Adj Close  \
Ticker Date                                                                
A      2015-03-17  41.520000  41.689999  41.060001  41.580002  38.292370   
       2015-03-18  41.450001  42.380001  41.169998  42.119999  38.789661   
       2015-03-19  42.049999  42.250000  41.820000  42.209999  38.872547   
       2015-03-20  42.389999  42.529999  42.200001  42.209999  38.872547   
       2015-03-23  42.119999  42.439999  42.000000  42.200001  38.863327   

                      Volume       RSI  RSI_EWMA  Middle Band  Upper Band  \
Ticker Date                                                                 
A      2015-03-17  14.224238 -0.503458 -0.000724    -0.346009   -0.351238   
       2015-03-18  14.074485 -0.294759  0.462353    -0.345888   -0.351086   
       2015-03-19  13.897736 -0.176084  0.534800    -0.345667   -0.350954   
       2015-03-20  14.655137 -0.442927  0.534800    -0.345612   -0.350852   
       2015-03-23  14.422774 -0.211576  0.518120    -0.345514   -0.350714   

                   ...  Signal Line  Histogram       ATR       RoC       OBV  \
Ticker Date        ...                                                         
A      2015-03-17  ...     0.000678  -0.083036 -0.604414 -0.315258  0.040657   
       2015-03-18  ...    -0.003270  -0.050854 -0.597702 -0.194259  0.040670   
       2015-03-19  ...    -0.005192  -0.026927 -0.599549 -0.118306  0.040681   
       2015-03-20  ...    -0.005920  -0.012832 -0.598741 -0.272145  0.040681   
       2015-03-23  ...    -0.006069  -0.006002 -0.599141 -0.137343  0.040663   

                     SMA_50    EMA_50  GK Volatility  Market Pct Change  \
Ticker Date                                                               
A      2015-03-17 -0.352246 -0.349892      -0.370571          -0.003320   
       2015-03-18 -0.352050 -0.349668       0.351412           0.012158   
       2015-03-19 -0.351794 -0.349437      -0.762746          -0.004873   
       2015-03-20 -0.351583 -0.349215      -0.964887           0.009013   
       2015-03-23 -0.351472 -0.349004      -0.718739          -0.001746   

                   Rolling Beta  
Ticker Date                      
A      2015-03-17      1.435489  
       2015-03-18      1.375154  
       2015-03-19      1.320370  
       2015-03-20      1.275087  
       2015-03-23      1.269052  

[5 rows x 22 columns]

In [236]:
from sklearn.preprocessing import MinMaxScaler

# Step 1: Compute Log Returns (Corrected)
df_transformed['Log Return'] = df_transformed.groupby(level='Ticker')['Adj Close'].pct_change(fill_method=None).apply(lambda x: np.log1p(x))

# Step 2: Compute Rolling Cumulative Returns (Momentum)
df_transformed['Quarterly Cumulative Return'] = df_transformed.groupby(level='Ticker')['Log Return']\
    .rolling(window=63, min_periods=30).sum().droplevel(0)  # Quaterly Momentum

# Step 3: Compute Rolling Volatility
df_transformed['Quarterly Volatility'] = df_transformed.groupby(level='Ticker')['Log Return']\
    .rolling(window=63, min_periods=30).std().droplevel(0)  # Quaterly Volatility

# Step 4: Compute Sharpe Ratio
df_transformed['Sharpe Ratio'] = df_transformed['Quarterly Cumulative Return'] / df_transformed['Quarterly Volatility']

# Step 5: Normalize Momentum & Sharpe Ratio for Ranking
scaler = MinMaxScaler()

# Drop NaNs before scaling
valid_idx = df_transformed[['Quarterly Cumulative Return', 'Sharpe Ratio']].dropna().index
momentum_sharpe_scaled = scaler.fit_transform(df_transformed.loc[valid_idx, ['Quarterly Cumulative Return', 'Sharpe Ratio']])

df_transformed.loc[valid_idx, 'Momentum Score'] = momentum_sharpe_scaled[:, 0]
df_transformed.loc[valid_idx, 'Sharpe Score'] = momentum_sharpe_scaled[:, 1]

# Step 6: Compute the Final Quaterly Blended Score
df_transformed['Quarterly Score'] = 0.5 * df_transformed['Momentum Score'] + 0.5 * df_transformed['Sharpe Score']

# Step 7: Select the Top 20 Stocks Based on Final Score

# Select the last available trading date in each quater
quarterly_end_dates = df_transformed.groupby(pd.Grouper(level='Date', freq='QE')).apply(lambda x: x.index.get_level_values('Date').max())
# quarterly_end_dates = pd.to_datetime(quarterly_end_dates)
# quarterly_end_dates = quarterly_end_dates.dt.strftime('%Y-%m-%d')  # Ensure string format
# print(quarterly_end_dates)

#Filter the dataset for only the end of quarter rows
quarterly_stocks = df_transformed.loc[df_transformed.index.get_level_values('Date').isin(quarterly_end_dates)]

# print(df_transformed.index.get_level_values('Date').isin(quaterly_end_dates))
# print(df_transformed.index.get_level_values('Date'))
# print(quaterly_end_dates)


# Select top 20 stocks per quarter
top_20_stocks_per_quater = quarterly_stocks.groupby('Date').apply(
    lambda x: x.sort_values(by='Quarterly Score', ascending=False).head(20)
)

top_20_stocks_per_quater

Open         High          Low  \
Date       Ticker Date                                                
2015-03-31 A      2015-03-31    41.279999    41.779999    41.099998   
           AAPL   2015-03-31    31.522499    31.622499    31.090000   
           ABBV   2015-03-31    58.009998    58.810001    57.709999   
           ABNB   2015-03-31          NaN          NaN          NaN   
           ABT    2015-03-31    46.830002    46.980000    46.330002   
...                                   ...          ...          ...   
2023-12-29 ALL    2023-12-29   139.830002   140.389999   139.410004   
           FICO   2023-12-29  1168.099976  1176.219971  1155.270020   
           STX    2023-12-29    86.570000    86.980003    85.169998   
           BK     2023-12-29    52.200001    52.299999    51.880001   
           AMT    2023-12-29   215.979996   216.800003   215.000000   

                                    Close    Adj Close     Volume       RSI  \
Date       Ticker Date                                                        
2015-03-31 A      2015-03-31    41.549999    38.358982  14.321847  0.290662   
           AAPL   2015-03-31    31.107500    27.819214  18.941629  0.110840   
           ABBV   2015-03-31    58.540001    38.630005  16.326215  0.396246   
           ABNB   2015-03-31          NaN          NaN        NaN       NaN   
           ABT    2015-03-31    46.330002    38.292351  15.494120 -0.160489   
...                                   ...          ...        ...       ...   
2023-12-29 ALL    2023-12-29   139.979996   137.112045  13.491719 -0.071045   
           FICO   2023-12-29  1164.010010  1164.010010  11.482477  0.520603   
           STX    2023-12-29    85.370003    82.942093  14.074640  0.921698   
           BK     2023-12-29    52.049999    50.302078  14.432371  1.476544   
           AMT    2023-12-29   215.880005   208.808929  14.275717  1.514969   

                              RSI_EWMA  Middle Band  Upper Band  ...  \
Date       Ticker Date                                           ...   
2015-03-31 A      2015-03-31  0.027879    -0.346873   -0.352078  ...   
           AAPL   2015-03-31 -0.457969    -0.392914   -0.396488  ...   
           ABBV   2015-03-31 -0.269536    -0.345630   -0.345671  ...   
           ABNB   2015-03-31       NaN          NaN         NaN  ...   
           ABT    2015-03-31 -0.879439    -0.344233   -0.350719  ...   
...                                ...          ...         ...  ...   
2023-12-29 ALL    2023-12-29  0.498767     0.098339    0.083460  ...   
           FICO   2023-12-29  0.608818     4.684475    4.558388  ...   
           STX    2023-12-29  0.550914    -0.157364   -0.150308  ...   
           BK     2023-12-29  1.113870    -0.298968   -0.298681  ...   
           AMT    2023-12-29  1.207339     0.402293    0.382940  ...   

                              GK Volatility  Market Pct Change  Rolling Beta  \
Date       Ticker Date                                                         
2015-03-31 A      2015-03-31      -0.356124          -0.008796      1.190009   
           AAPL   2015-03-31      -0.571204          -0.008796      0.953383   
           ABBV   2015-03-31      -0.232925          -0.008796      0.730786   
           ABNB   2015-03-31            NaN          -0.008796           NaN   
           ABT    2015-03-31      -0.721321          -0.008796      1.035160   
...                                     ...                ...           ...   
2023-12-29 ALL    2023-12-29      -0.957146          -0.002826      0.411514   
           FICO   2023-12-29      -0.190195          -0.002826      0.703774   
           STX    2023-12-29      -0.234835          -0.002826      1.748306   
           BK     2023-12-29      -0.906228          -0.002826      1.298788   
           AMT    2023-12-29      -0.858393          -0.002826      0.656853   

                              Log Return  Quarterly Cumulative Return  \
Date       Ticker Date            

### Align historical data with political trade data

In [237]:
quarterly_aggregated = quarterly_aggregated[quarterly_aggregated['Quarter'] > '2019Q4']
quarterly_aggregated.head()

,Ticker,Quarter,Total_Transactions,Total_Buy_Amount,Total_Sell_Amount,Net_Buy_Sell,Buy_Sell_Ratio,Log_Buy_Sell_Ratio,Z_Score,Signal,Confidence
5,AAPL,2020Q1,32,1011000.0,7113501.0,-6102501.0,0.142124,-1.951055,-0.253835,Weak Sell - Hold,0.253835
6,AAPL,2020Q2,25,4679500.0,3129000.0,1550500.0,1.495526,0.402478,-0.042479,Weak Sell - Hold,0.042479
7,AAPL,2020Q3,21,41501.0,315500.0,-273999.0,0.131540,-2.028441,-0.260784,Weak Sell - Hold,0.260784
8,AAPL,2020Q4,18,170000.0,1254500.0,-1084500.0,0.135512,-1.998694,-0.258113,Weak Sell - Hold,0.258113
9,AAPL,2021Q1,21,223000.0,352500.0,-129500.0,0.632624,-0.457879,-0.119742,Weak Sell - Hold,0.119742


In [238]:
quarterly_aggregated['Quarter'] = quarterly_aggregated['Quarter'].dt.to_timestamp(how='end').dt.strftime('%Y-%m-%d')

In [239]:
df = top_20_stocks_per_quater.droplevel(level=2)
df.head()

Open       High        Low      Close  Adj Close  \
Date       Ticker                                                          
2015-03-31 A       41.279999  41.779999  41.099998  41.549999  38.358982   
           AAPL    31.522499  31.622499  31.090000  31.107500  27.819214   
           ABBV    58.009998  58.810001  57.709999  58.540001  38.630005   
           ABNB          NaN        NaN        NaN        NaN        NaN   
           ABT     46.830002  46.980000  46.330002  46.330002  38.292351   

                      Volume       RSI  RSI_EWMA  Middle Band  Upper Band  \
Date       Ticker                                                           
2015-03-31 A       14.321847  0.290662  0.027879    -0.346873   -0.352078   
           AAPL    18.941629  0.110840 -0.457969    -0.392914   -0.396488   
           ABBV    16.326215  0.396246 -0.269536    -0.345630   -0.345671   
           ABNB          NaN       NaN       NaN          NaN         NaN   
           ABT     15.494120 -0.160489 -0.879439    -0.344233   -0.350719   

                   ...  GK Volatility  Market Pct Change  Rolling Beta  \
Date       Ticker  ...                                                   
2015-03-31 A       ...      -0.356124          -0.008796      1.190009   
           AAPL    ...      -0.571204          -0.008796      0.953383   
           ABBV    ...      -0.232925          -0.008796      0.730786   
           ABNB    ...            NaN          -0.008796           NaN   
           ABT     ...      -0.721321          -0.008796      1.035160   

                   Log Return  Quarterly Cumulative Return  \
Date       Ticker                                            
2015-03-31 A        -0.004083                          NaN   
           AAPL     -0.015471                          NaN   
           ABBV      0.005138                          NaN   
           ABNB           NaN                          NaN   
           ABT      -0.016058                          NaN   

                   Quarterly Volatility  Sharpe Ratio  Momentum Score  \
Date       Ticker                                                       
2015-03-31 A                        NaN           NaN             NaN   
           AAPL                     NaN           NaN             NaN   
           ABBV                     NaN           NaN             NaN   
           ABNB                     NaN           NaN             NaN   
           ABT                      NaN           NaN             NaN   

                   Sharpe Score  Quarterly Score  
Date       Ticker                                 
2015-03-31 A                NaN              NaN  
           AAPL             NaN              NaN  
           ABBV             NaN              NaN  
           ABNB             NaN              NaN  
           ABT              NaN              NaN  

[5 rows x 29 columns]

In [240]:
df = df[['Adj Close', 'Log Return', 'Momentum Score', 'Sharpe Score', 'Quarterly Score']]
quarterly_aggregated = quarterly_aggregated[['Ticker', 'Quarter', 'Total_Transactions', 'Signal', 'Confidence']]
quarterly_aggregated = quarterly_aggregated.rename(columns={'Quarter': 'Date'})
quarterly_aggregated = quarterly_aggregated.set_index(['Ticker', 'Date'])
quarterly_aggregated.head()

Total_Transactions            Signal  Confidence
Ticker Date                                                        
AAPL   2020-03-31                  32  Weak Sell - Hold    0.253835
       2020-06-30                  25  Weak Sell - Hold    0.042479
       2020-09-30                  21  Weak Sell - Hold    0.260784
       2020-12-31                  18  Weak Sell - Hold    0.258113
       2021-03-31                  21  Weak Sell - Hold    0.119742

In [241]:
quarterly_aggregated

Total_Transactions            Signal  Confidence
Ticker Date                                                        
AAPL   2020-03-31                  32  Weak Sell - Hold    0.253835
       2020-06-30                  25  Weak Sell - Hold    0.042479
       2020-09-30                  21  Weak Sell - Hold    0.260784
       2020-12-31                  18  Weak Sell - Hold    0.258113
       2021-03-31                  21  Weak Sell - Hold    0.119742
...                               ...               ...         ...
XOM    2020-03-31                  20   Weak Buy - Hold    0.036089
       2020-06-30                   6  Weak Sell - Hold    0.078623
       2021-03-31                   8   Weak Buy - Hold    0.010158
       2021-06-30                   5        Strong Buy    2.156582
       2022-12-31                   6  Weak Sell - Hold    0.512700

[430 rows x 3 columns]

In [242]:
df = df.reset_index()
df = df[df['Date'] >= '2020-03-31']
df = df.set_index(['Ticker', 'Date'])
df

,,Adj Close,Log Return,Momentum Score,Sharpe Score,Quarterly Score
Ticker,Date,,,,,
MRNA,2020-03-31,29.950001,-0.017541,0.650192,0.541254,0.595723
GEN,2020-03-31,16.850950,-0.020630,0.596705,0.586172,0.591439
REGN,2020-03-31,488.290009,0.031814,0.602313,0.561669,0.581991
DXCM,2020-03-31,67.317497,-0.031907,0.589916,0.529900,0.559908
TSLA,2020-03-31,34.933334,0.042633,0.593638,0.502256,0.547947
...,...,...,...,...,...,...
ALL,2023-12-29,137.112045,0.001144,0.593868,0.698809,0.646339
FICO,2023-12-29,1164.010010,-0.004569,0.608878,0.682338,0.645608
STX,2023-12-29,82.942093,-0.016497,0.602059,0.687494,0.644777


In [245]:
quarterly_aggregated.head()

Total_Transactions            Signal  Confidence
Ticker Date                                                        
AAPL   2020-03-31                  32  Weak Sell - Hold    0.253835
       2020-06-30                  25  Weak Sell - Hold    0.042479
       2020-09-30                  21  Weak Sell - Hold    0.260784
       2020-12-31                  18  Weak Sell - Hold    0.258113
       2021-03-31                  21  Weak Sell - Hold    0.119742

In [247]:
df_combined = quarterly_aggregated.join(df, how='right')
df_combined.head(100)

,,Total_Transactions,Signal,Confidence,Adj Close,Log Return,Momentum Score,Sharpe Score,Quarterly Score
Ticker,Date,,,,,,,,
MRNA,2020-03-31,NaN,NaN,NaN,29.950001,-0.017541,0.650192,0.541254,0.595723
GEN,2020-03-31,NaN,NaN,NaN,16.850950,-0.020630,0.596705,0.586172,0.591439
REGN,2020-03-31,NaN,NaN,NaN,488.290009,0.031814,0.602313,0.561669,0.581991
DXCM,2020-03-31,NaN,NaN,NaN,67.317497,-0.031907,0.589916,0.529900,0.559908
TSLA,2020-03-31,NaN,NaN,NaN,34.933334,0.042633,0.593638,0.502256,0.547947
...,...,...,...,...,...,...,...,...,...
MHK,2021-03-31,NaN,NaN,NaN,192.309998,-0.014045,0.616237,0.663527,0.639882
URI,2021-03-31,NaN,NaN,NaN,321.792999,-0.011773,0.630032,0.645591,0.637812
XOM,2021-03-31,NaN,NaN,NaN,48.108475,-0.015286,0.615656,0.656695,0.636175
